In [1]:
import os

import ir_datasets
import evaluate
from tqdm.notebook import tqdm

from irise import INDEX_DIR
from irise.indexer import Indexer
from irise.pipeline import SearchPipeline

2024-06-06 23:04:18.909912: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 23:04:19.551260: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
SYSTEM = "TFIDF"
dataset = ir_datasets.load("beir/msmarco/test")
pipe = SearchPipeline()

/home/devrim/anaconda3/envs/irise/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
queries = [q for q in dataset.queries_iter()]

In [5]:
queries[24]

GenericQuery(query_id='855410', text='what is theraderm used for')

In [ ]:
predictions = {
    "query": [],
    "q0": [],
    "docid": [],
    "score": [],
    "rank": [],
    "system": [],
}

for query in tqdm(dataset.queries_iter(), total=dataset.queries_count()):
    if query.query_id == "855410":
        continue
    results = pipe(query.text, return_dict=False)
    if not results:
        predictions["query"].append(int(query.query_id))
        predictions["q0"].append("q0")
        predictions["docid"].append(str(-1))
        predictions["score"].append(-1)
        predictions["rank"].append(-1)
        predictions["system"].append(SYSTEM)
    else:
        for rank, (doc_id, score) in enumerate(results):
            predictions["query"].append(int(query.query_id))
            predictions["q0"].append("q")
            predictions["docid"].append(doc_id)
            predictions["score"].append(score)
            predictions["rank"].append(rank)
            predictions["system"].append(SYSTEM)

  0%|          | 0/43 [00:00<?, ?it/s]

In [ ]:
qrels = {
    "query": [],
    "q0": [],
    "docid": [],
    "rel": [],
}
for qrel in tqdm(dataset.qrels_iter(), total=dataset.qrels_count()):
    if qrel.query_id == "855410":
        continue
    qrels["query"].append(int(qrel.query_id))
    qrels["q0"].append("q0")
    qrels["docid"].append(qrel.doc_id)
    qrels["rel"].append(qrel.relevance)

In [ ]:
trec_eval = evaluate.load("trec_eval")

In [ ]:
results_tfidf = trec_eval.compute(references=[qrels], predictions=[predictions])

In [ ]:
results_tfidf

In [34]:
results_tfidf

{'runid': 'TFIDF',
 'num_ret': 430,
 'num_rel': 4102,
 'num_rel_ret': 353,
 'num_q': 43,
 'map': 0.14863877029136405,
 'gm_map': 0.10623112713697273,
 'bpref': 0.15655697050989456,
 'Rprec': 0.15928905616616532,
 'recip_rank': 0.9593023255813954,
 'P@5': 0.8790697674418604,
 'P@10': 0.8209302325581395,
 'P@15': 0.5472868217054264,
 'P@20': 0.41046511627906973,
 'P@30': 0.2736434108527132,
 'P@100': 0.08209302325581395,
 'P@200': 0.04104651162790698,
 'P@500': 0.01641860465116279,
 'P@1000': 0.008209302325581395,
 'NDCG@5': 0.729520788480906,
 'NDCG@10': 0.7064472702743727,
 'NDCG@15': 0.5765216412900166,
 'NDCG@20': 0.5030617383987054,
 'NDCG@30': 0.42685434713660225,
 'NDCG@100': 0.3201343402421015,
 'NDCG@200': 0.30607353920002395,
 'NDCG@500': 0.30394912747731856,
 'NDCG@1000': 0.30394912747731856}